# Rodando em simulador e maquina quantica

In [1]:
from qiskit.circuit.library import PauliTwoDesign, TwoLocal, EfficientSU2, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP, NFT
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA, SamplingVQE
from qiskit.quantum_info import Pauli

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Sampler

from IPython.display import clear_output

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd
import pickle
import time

In [2]:
vqe_4_local = pd.DataFrame()
vqe_10_local = pd.DataFrame()
qaoa_4_local = pd.DataFrame()
qaoa_10_local = pd.DataFrame()

with open('pickles/vqe_4_local.pickle', 'rb') as file:
    vqe_4_local = pickle.load(file)

with open('pickles/vqe_10_local.pickle', 'rb') as file:
    vqe_10_local = pickle.load(file)

with open('pickles/qaoa_4_local.pickle', 'rb') as file:
    qaoa_4_local = pickle.load(file)

with open('pickles/qaoa_10_local.pickle', 'rb') as file:
    qaoa_10_local = pickle.load(file)


programa_quadratico_4 = None
programa_quadratico_10 = None

with open('pickles/programa_quadratico_4.pickle', 'rb') as file:
    programa_quadratico_4 = pickle.load(file)

with open('pickles/programa_quadratico_10.pickle', 'rb') as file:
    programa_quadratico_10 = pickle.load(file)

In [3]:
print(len(vqe_4_local))
print(len(vqe_10_local))
print(len(qaoa_4_local))
print(len(qaoa_10_local))

245290
406180
151438
148972


In [3]:
indices_epocas_finais_vqe_4_local = vqe_4_local.loc[vqe_4_local.nova_epoca == True].groupby(['ansatz', 'otimizador'])['epoca'].idxmax()
resultados_vqe_4 = vqe_4_local.loc[indices_epocas_finais_vqe_4_local, ['parametros', 'valor_otimo_encontrado', 'tempo_elapsado_otimizador', 'probabilidade_valor_otimo', 'amostras_otimizacao', 'ansatz', 'otimizador']]
resultados_vqe_4 = resultados_vqe_4.set_index(['otimizador', 'ansatz'])

print(resultados_vqe_4.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
resultados_vqe_4

\begin{tabular}{lllrrrl}
\toprule
 &  & parametros & valor_otimo_encontrado & tempo_elapsado_otimizador & probabilidade_valor_otimo & amostras_otimizacao \\
otimizador & ansatz &  &  &  &  &  \\
\midrule
COBYLA & EfficientSU2 & [-1.5142979390811262, 2.307387606411415, 1.7593651952090907, 3.2045207134717297, 0.138595024843831, -1.2092634968003815, 0.46152527691706924, 7.298676292019879, -1.2555300753671834, 1.8463183368337082, -2.5835153385307303, 0.48008352961746104, -0.8021802468682963, 0.7520126075860815, 1.5821903225555867, -1.0261830063841666, 1.8299245980332877, 0.33110737415399466, 1.5157391501388504, 3.215638847110921, -1.0360855390103771, -0.3919268859851479, 2.3506400735677415, -0.5993615398158165, -0.21939974577047602, -3.2726311531368037, 0.6805755474810505, 3.1733276789581604, 12.566492691958127, 6.826104086382959, 9.860008270738005, 8.0097842697476] & -1.8 & 0.9 & 0.9 & [SolutionSample(x=array([1., 0., 1., 0.]), fval=-0.5274735324342604, probability=0.87890625, status=<Opt

,,parametros,valor_otimo_encontrado,tempo_elapsado_otimizador,probabilidade_valor_otimo,amostras_otimizacao
otimizador,ansatz,,,,,
COBYLA,EfficientSU2,"[-1.5142979390811262, 2.307387606411415, 1.759...",-1.816578,0.946133,0.932617,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
L_BFGS_B,EfficientSU2,"[0.49936624880039365, 1.9533293316304563, 2.39...",-1.072015,3.667540,0.097656,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
NFT,EfficientSU2,"[3.118488217948747, -1.4596659418871876, 1.632...",-1.904453,3.149668,0.999023,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
SLSQP,EfficientSU2,"[1.498474392455133, -0.7061383139985018, -1.44...",-1.202363,0.101660,0.007812,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
COBYLA,PauliTwoDesign,"[1.0555042795323522, 1.4064360565941822, 1.877...",-1.664922,0.472108,0.015625,"[SolutionSample(x=array([0., 0., 1., 1.]), fva..."
L_BFGS_B,PauliTwoDesign,"[3.4648036340805257, -0.0755097161375413, -5.3...",-0.962966,1.495825,0.013672,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
NFT,PauliTwoDesign,"[-5.892166807257425, -1.6385778915958227, 1.61...",-1.750365,3.828763,0.721680,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
SLSQP,PauliTwoDesign,"[-9323229.914351651, 12497636.437782174, 15129...",-0.411333,6.338441,0.832031,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
COBYLA,RealAmplitudes,"[0.12927802779352743, 2.8589781584164125, 1.98...",-1.822332,0.469526,0.138672,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."


In [4]:
indices_epocas_finais_vqe_10_local = vqe_10_local.loc[vqe_10_local.nova_epoca == True].groupby(['ansatz', 'otimizador'])['epoca'].idxmax()
resultados_vqe_10 = vqe_10_local.loc[indices_epocas_finais_vqe_10_local, ['parametros', 'valor_otimo_encontrado', 'tempo_elapsado_otimizador', 'probabilidade_valor_otimo', 'amostras_otimizacao', 'ansatz', 'otimizador']]
resultados_vqe_10 = resultados_vqe_10.set_index(['otimizador', 'ansatz'])

print(resultados_vqe_10.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
resultados_vqe_10

\begin{tabular}{lllrrrl}
\toprule
 &  & parametros & valor_otimo_encontrado & tempo_elapsado_otimizador & probabilidade_valor_otimo & amostras_otimizacao \\
otimizador & ansatz &  &  &  &  &  \\
\midrule
COBYLA & EfficientSU2 & [-0.2270777293663846, -0.10238983467987119, 0.7944875884019648, 1.51376845844082, 0.7513116914232977, 3.1957544685417307, 3.0576753778635295, 0.0012652036007215618, 2.360144856564227, 0.7508135125147353, 17.031668557889542, 8.003824153398591, 2.991855553750596, 0.04459758476864733, 3.438172258714555, 1.4045019066427678, 0.3862298196781958, 2.9663680722937333, -0.32676489924615193, 1.0857245218005553, 1.2160182882372632, 3.082850180802574, -1.5682876082150254, -1.182782761356467, -0.8829909913127925, 1.1161429568924734, 0.4826905137933452, 0.1620634590929346, 0.8965825928045538, -1.9146621551582592, 3.3144252431283174, 2.4724394176726703, -0.11968869513260352, -2.3873404054844065, 2.8390536556269153, 3.1199825705044946, 3.4285137448436704, 1.403208923421207, -2.4

,,parametros,valor_otimo_encontrado,tempo_elapsado_otimizador,probabilidade_valor_otimo,amostras_otimizacao
otimizador,ansatz,,,,,
COBYLA,EfficientSU2,"[-0.2270777293663846, -0.10238983467987119, 0....",-46.934902,3.442080,0.006836,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
L_BFGS_B,EfficientSU2,"[-0.8047468390122801, -0.12016921654425626, 0....",-24.327851,27.674974,0.015625,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
NFT,EfficientSU2,"[0.01483559902348891, -0.019151790475619747, 3...",-48.340793,7.409147,0.000977,"[SolutionSample(x=array([0., 0., 1., 0., 0., 0..."
SLSQP,EfficientSU2,"[-1.3594828016204732, 2.2912223276736796, 3.05...",-13.569150,4.865007,0.023438,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
COBYLA,PauliTwoDesign,"[-1.3401285084564858, 8.535304210668336, -0.77...",-46.684925,2.070838,0.000977,"[SolutionSample(x=array([0., 0., 0., 0., 0., 1..."
L_BFGS_B,PauliTwoDesign,"[-4.058126159261859, 2.5910856700934275, -1.89...",-26.203385,7.262081,0.007812,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
NFT,PauliTwoDesign,"[1.379128634747774, 6.712970371958819, 2.27127...",-47.239329,6.502862,0.000977,"[SolutionSample(x=array([0., 0., 1., 0., 0., 0..."
SLSQP,PauliTwoDesign,"[736394210.7922246, 452396271.0204296, 3418250...",-8.360271,41.151175,0.030273,"[SolutionSample(x=array([0., 0., 1., 0., 0., 0..."
COBYLA,RealAmplitudes,"[-2.920078802098587, -0.2171149432375934, -3.2...",-48.091788,1.703619,0.100586,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."


In [5]:
indices_epocas_finais_qaoa_4_local = qaoa_4_local.loc[qaoa_4_local.nova_epoca == True].groupby(['mixer', 'otimizador'])['epoca'].idxmax()
resultados_qaoa_4 = qaoa_4_local.loc[indices_epocas_finais_qaoa_4_local, ['parametros', 'valor_otimo_encontrado', 'tempo_elapsado_otimizador', 'probabilidade_valor_otimo', 'amostras_otimizacao', 'mixer', 'otimizador']]
resultados_qaoa_4 = resultados_qaoa_4.set_index(['otimizador', 'mixer'])

print(resultados_qaoa_4.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
resultados_qaoa_4

\begin{tabular}{lllrrrl}
\toprule
 &  & parametros & valor_otimo_encontrado & tempo_elapsado_otimizador & probabilidade_valor_otimo & amostras_otimizacao \\
otimizador & mixer &  &  &  &  &  \\
\midrule
COBYLA & full_mixer & [7.200935379710995, -5.905469512861117] & -0.1 & 0.2 & 0.1 & [SolutionSample(x=array([1., 0., 1., 0.]), fval=-0.5274735324342604, probability=0.0693359375, status=<OptimizationResultStatus.SUCCESS: 0>), SolutionSample(x=array([0., 0., 1., 1.]), fval=-0.45428006682762206, probability=0.0625, status=<OptimizationResultStatus.SUCCESS: 0>), SolutionSample(x=array([0., 1., 1., 0.]), fval=-0.4464891778629332, probability=0.0673828125, status=<OptimizationResultStatus.SUCCESS: 0>), SolutionSample(x=array([1., 0., 0., 1.]), fval=-0.31555074247208253, probability=0.0625, status=<OptimizationResultStatus.SUCCESS: 0>), SolutionSample(x=array([1., 1., 0., 0.]), fval=-0.3077054949407142, probability=0.052734375, status=<OptimizationResultStatus.SUCCESS: 0>), SolutionSample(x=ar

,,parametros,valor_otimo_encontrado,tempo_elapsado_otimizador,probabilidade_valor_otimo,amostras_otimizacao
otimizador,mixer,,,,,
COBYLA,full_mixer,"[7.200935379710995, -5.905469512861117]",-0.075302,0.182393,0.108398,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
L_BFGS_B,full_mixer,"[-2.193283203540099, -5.919583952905944]",0.008450,0.318933,0.150391,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
NFT,full_mixer,"[220.56057128564484, 80.24507270495481]",0.174098,4.842583,0.141602,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
SLSQP,full_mixer,"[12881064.979279414, -9012350.491580186]",-0.057253,0.711714,0.126953,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
COBYLA,mixer_default,"[3.3654652812587003, -3.849155850623404]",-0.775968,0.229233,0.121094,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
L_BFGS_B,mixer_default,"[3.3600878034143027, 3.184892227968962]",-0.787206,0.320539,0.100586,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
NFT,mixer_default,"[82.37547968649578, 40.91973425342345]",0.033338,5.459131,0.900391,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
SLSQP,mixer_default,"[-1441015.8589012334, -5190740.957774206]",0.406204,0.652428,0.242188,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."
COBYLA,parity_ring_mixer,"[5.6291616924156855, 0.23476082961507866]",-0.907475,0.160004,0.083984,"[SolutionSample(x=array([1., 0., 1., 0.]), fva..."


In [6]:
indices_epocas_finais_qaoa_10_local = qaoa_10_local.loc[qaoa_10_local.nova_epoca == True].groupby(['mixer', 'otimizador'])['epoca'].idxmax()
resultados_qaoa_10 = qaoa_10_local.loc[indices_epocas_finais_qaoa_10_local, ['parametros', 'valor_otimo_encontrado', 'tempo_elapsado_otimizador', 'probabilidade_valor_otimo', 'amostras_otimizacao', 'mixer', 'otimizador']]
resultados_qaoa_10 = resultados_qaoa_10.set_index(['otimizador', 'mixer'])

print(resultados_qaoa_10.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.1f}".format,
))
resultados_qaoa_10

\begin{tabular}{lllrrrl}
\toprule
 &  & parametros & valor_otimo_encontrado & tempo_elapsado_otimizador & probabilidade_valor_otimo & amostras_otimizacao \\
otimizador & mixer &  &  &  &  &  \\
\midrule
COBYLA & full_mixer & [8.299308010068483, 5.987369958933673] & -5.7 & 0.6 & 0.0 & [SolutionSample(x=array([0., 1., 1., 0., 0., 0., 0., 0., 0., 0.]), fval=-1.6139113787680603, probability=0.0029296875, status=<OptimizationResultStatus.SUCCESS: 0>), SolutionSample(x=array([0., 0., 1., 0., 0., 1., 0., 0., 0., 0.]), fval=-1.5454296263278404, probability=0.0029296875, status=<OptimizationResultStatus.SUCCESS: 0>), SolutionSample(x=array([1., 0., 1., 0., 0., 0., 0., 0., 0., 0.]), fval=-1.536257170820941, probability=0.0048828125, status=<OptimizationResultStatus.SUCCESS: 0>), SolutionSample(x=array([0., 0., 1., 0., 1., 0., 0., 0., 0., 0.]), fval=-1.5232300893466808, probability=0.0009765625, status=<OptimizationResultStatus.SUCCESS: 0>), SolutionSample(x=array([0., 0., 1., 0., 0., 0., 0., 0.,

,,parametros,valor_otimo_encontrado,tempo_elapsado_otimizador,probabilidade_valor_otimo,amostras_otimizacao
otimizador,mixer,,,,,
COBYLA,full_mixer,"[8.299308010068483, 5.987369958933673]",-5.704022,0.606503,0.002930,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
L_BFGS_B,full_mixer,"[5.629624777484993, 0.2618947584002477]",-21.856512,1.027055,0.004883,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
NFT,full_mixer,"[41.543137535550194, -0.013570924556014285]",-1.959224,20.115099,0.005859,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
SLSQP,full_mixer,"[-388583300.43488634, 99052556.3815338]",2.873818,1.992957,0.002930,"[SolutionSample(x=array([0., 0., 1., 1., 0., 0..."
COBYLA,mixer_default,"[-3.6948828600734145, -3.7819713546267235]",-34.526558,0.564184,0.037109,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
L_BFGS_B,mixer_default,"[3.977651891456843, 5.956097380553876]",-44.136478,2.242312,0.008789,"[SolutionSample(x=array([0., 0., 1., 0., 0., 0..."
NFT,mixer_default,"[19.603069923471473, 7.442840629346817]",-3.558936,18.809813,0.018555,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
SLSQP,mixer_default,"[-2742069.0044090347, -3466907.8463931014]",2.535267,5.358600,0.003906,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."
COBYLA,parity_ring_mixer,"[5.543972601915874, -2.2621240443797013]",-9.064221,0.610968,0.003906,"[SolutionSample(x=array([0., 1., 1., 0., 0., 0..."


In [7]:
service = QiskitRuntimeService(channel="ibm_quantum")

ambiente_backend_simulador ="ibmq_qasm_simulator"
ambiente_backend_computador_quantico ="ibm_brisbane"

backend_simulador = service.backend(ambiente_backend_simulador)
backend_computador_quantico = service.backend(ambiente_backend_computador_quantico)

sampler_simulador = Sampler(backend=backend_simulador)
sampler_computador_quantico = Sampler(backend=backend_computador_quantico)

seed=123

## Roda para 4 qubits

In [8]:
num_qubits = 4

In [9]:
# Function to create a ring mixer
def create_ring_mixer(num_qubits):
    ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        ring_mixer *= Pauli(''.join(pauli_list))
    return ring_mixer

# Function to create a parity ring mixer
def create_parity_ring_mixer(num_qubits):
    parity_ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        parity_ring_mixer *= Pauli(''.join(pauli_list))
    pauli_list[-1] = 'X'  # Apply X gate on the last qubit
    parity_ring_mixer *= Pauli(''.join(pauli_list))
    return parity_ring_mixer

# Function to create a full mixer
def create_full_mixer(num_qubits):
    return Pauli('X'*num_qubits)

ring_mixer = create_ring_mixer(num_qubits)
full_mixer = create_full_mixer(num_qubits)
parity_ring_mixer = create_parity_ring_mixer(num_qubits)

otimizadores = {'NFT': NFT(maxiter=100),
                'SLSQP': SLSQP(maxiter=100),
                'COBYLA': COBYLA(maxiter=100),
                'L_BFGS_B': L_BFGS_B(maxiter=100)}

mixers = {'mixer_default': None, 
            'ring_mixer': ring_mixer,
            'full_mixer': full_mixer,
            'parity_ring_mixer': parity_ring_mixer}

ansatzes = {'EfficientSU2': EfficientSU2(num_qubits),
            'RealAmplitudes': RealAmplitudes(num_qubits),
            'PauliTwoDesign': PauliTwoDesign(num_qubits, reps=3),
            'TwoLocal': TwoLocal(num_qubits, "ry", "cz", reps=3, entanglement="full")}

## VQE

In [ ]:
resultados_vqe_4['parametros_simulador'] = None
resultados_vqe_4['amostras_otimizacao_simulador'] = None
resultados_vqe_4['valor_otimo_encontrado_simulador'] = None
resultados_vqe_4['tempo_elapsado_otimizador_simulador'] = None
resultados_vqe_4['probabilidade_valor_otimo_simulador'] = None

resultados_vqe_4['parametros_computador_quantico'] = None
resultados_vqe_4['amostras_otimizacao_computador_quantico'] = None
resultados_vqe_4['valor_otimo_encontrado_computador_quantico'] = None
resultados_vqe_4['tempo_elapsado_otimizador_computador_quantico'] = None
resultados_vqe_4['probabilidade_valor_otimo_computador_quantico'] = None

for indice, row in resultados_vqe_4.iterrows():

    ansatz = ansatzes[indice[1]]
    otimizador = otimizadores[indice[0]]
    parametros_iniciais = row['parametros']

    lista_energia_aux = []
    lista_parametros_aux = []

    def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
        lista_parametros_aux.append(parameters)
        lista_energia_aux.append(mean)
        time.sleep(5)
        clear_output(wait=False)

    vqe_simulador = SamplingVQE(sampler=sampler_simulador, ansatz=ansatz, optimizer=otimizador, initial_point=parametros_iniciais, callback=guardar_parametros_chamada_custo)
    vqe_simulador.random_seed = seed
    vqe_eigen_simulador = MinimumEigenOptimizer(vqe_simulador)
    resultado_simulador = vqe_eigen_simulador.solve(programa_quadratico_4)

    indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
    parametros_simulador = lista_parametros_aux[indice_menor_energia]

    objeto_resultado_simulador = resultado_simulador.min_eigen_solver_result

    amostras_otimizacao_simulador = resultado_simulador.samples
    valor_otimo_encontrado_simulador = objeto_resultado_simulador.optimal_value
    tempo_elapsado_otimizador_simulador = objeto_resultado_simulador.optimizer_time
    probabilidade_valor_otimo_simulador = objeto_resultado_simulador.best_measurement['probability']

    resultados_vqe_4.at[indice, 'parametros_simulador'] = parametros_simulador
    resultados_vqe_4.at[indice, 'amostras_otimizacao_simulador'] = amostras_otimizacao_simulador
    resultados_vqe_4.at[indice, 'valor_otimo_encontrado_simulador'] = valor_otimo_encontrado_simulador
    resultados_vqe_4.at[indice, 'tempo_elapsado_otimizador_simulador'] = tempo_elapsado_otimizador_simulador
    resultados_vqe_4.at[indice, 'probabilidade_valor_otimo_simulador'] = probabilidade_valor_otimo_simulador

    with open('pickles/resultados_vqe_4_diferentes_metodos_parcial_s.pickle', 'wb') as file:
        pickle.dump(resultados_vqe_4, file)

# for indice, row in resultados_vqe_4.iterrows():

#     ansatz = ansatzes[indice[1]]
#     otimizador = otimizadores[indice[0]]
#     parametros_iniciais = row['parametros']

#     lista_energia_aux = []
#     lista_parametros_aux = []

#     def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
#         lista_parametros_aux.append(parameters)
#         lista_energia_aux.append(mean)
#         clear_output(wait=False)

#     vqe_computador_quantico = SamplingVQE(sampler=sampler_computador_quantico, ansatz=ansatz, optimizer=otimizador, initial_point=parametros_iniciais, callback=guardar_parametros_chamada_custo)
#     vqe_computador_quantico.random_seed = seed
#     vqe_eigen_computador_quantico = MinimumEigenOptimizer(vqe_computador_quantico)
#     resultado_computador_quantico = vqe_eigen_computador_quantico.solve(programa_quadratico_4)

#     indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
#     parametros_computador_quantico = lista_parametros_aux[indice_menor_energia]

#     objeto_resultado_computador_quantico = resultado_computador_quantico.min_eigen_solver_result

#     amostras_otimizacao_computador_quantico = resultado_computador_quantico.samples
#     valor_otimo_encontrado_computador_quantico = objeto_resultado_computador_quantico.optimal_value
#     tempo_elapsado_otimizador_computador_quantico = objeto_resultado_computador_quantico.optimizer_time
#     probabilidade_valor_otimo_computador_quantico = objeto_resultado_computador_quantico.best_measurement['probability']

#     resultados_vqe_4.at[indice, 'parametros_computador_quantico'] = parametros_computador_quantico
#     resultados_vqe_4.at[indice, 'amostras_otimizacao_computador_quantico'] = amostras_otimizacao_computador_quantico
#     resultados_vqe_4.at[indice, 'valor_otimo_encontrado_computador_quantico'] = valor_otimo_encontrado_computador_quantico
#     resultados_vqe_4.at[indice, 'tempo_elapsado_otimizador_computador_quantico'] = tempo_elapsado_otimizador_computador_quantico
#     resultados_vqe_4.at[indice, 'probabilidade_valor_otimo_computador_quantico'] = probabilidade_valor_otimo_computador_quantico

#     with open('pickles/resultados_vqe_4_diferentes_metodos_parcial_q.pickle', 'wb') as file:
#         pickle.dump(resultados_vqe_4, file)

with open('pickles/resultados_vqe_4_diferentes_metodos.pickle', 'wb') as file:
    pickle.dump(resultados_vqe_4, file)

resultados_vqe_4

In [19]:
copia_resultados_vqe_4 = resultados_vqe_4.copy()

combinacoes = [f"{otimizador} e {ansatz}" for otimizador, ansatz in copia_resultados_vqe_4.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=False, shared_yaxes=False)

for i, (otimizador, ansatz) in enumerate(copia_resultados_vqe_4.index.unique()):

    df_subset = copia_resultados_vqe_4.loc[
        (copia_resultados_vqe_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_vqe_4.index.get_level_values(1) == ansatz)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{ansatz}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/vqe_4_comparacao_metodos.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [20]:
copia_resultados_vqe_4 = resultados_vqe_4.copy()

combinacoes = [f"{otimizador} e {ansatz}" for otimizador, ansatz in copia_resultados_vqe_4.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=True, shared_yaxes=True)

for i, (otimizador, ansatz) in enumerate(copia_resultados_vqe_4.index.unique()):

    df_subset = copia_resultados_vqe_4.loc[
        (copia_resultados_vqe_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_vqe_4.index.get_level_values(1) == ansatz)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{ansatz}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/vqe_4_comparacao_metodos_mesmo_y.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

## QAOA

In [12]:
resultados_qaoa_4['parametros_simulador'] = None
resultados_qaoa_4['amostras_otimizacao_simulador'] = None
resultados_qaoa_4['valor_otimo_encontrado_simulador'] = None
resultados_qaoa_4['tempo_elapsado_otimizador_simulador'] = None
resultados_qaoa_4['probabilidade_valor_otimo_simulador'] = None

resultados_qaoa_4['parametros_computador_quantico'] = None
resultados_qaoa_4['amostras_otimizacao_computador_quantico'] = None
resultados_qaoa_4['valor_otimo_encontrado_computador_quantico'] = None
resultados_qaoa_4['tempo_elapsado_otimizador_computador_quantico'] = None
resultados_qaoa_4['probabilidade_valor_otimo_computador_quantico'] = None

for indice, row in resultados_qaoa_4.iterrows():
    print(indice)
    mixer = mixers[indice[1]]
    otimizador = otimizadores[indice[0]]
    parametros_iniciais = row['parametros']

    lista_energia_aux = []
    lista_parametros_aux = []

    def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
        lista_parametros_aux.append(parameters)
        lista_energia_aux.append(mean)
        time.sleep(5)
        clear_output(wait=False)

    qaoa = QAOA(sampler=sampler_simulador, optimizer=otimizador, reps=1, mixer=mixer, initial_point=parametros_iniciais, callback=guardar_parametros_chamada_custo)
    qaoa_eigen = MinimumEigenOptimizer(qaoa)
    resultado_simulador = qaoa_eigen.solve(programa_quadratico_4)

    indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
    parametros_simulador = lista_parametros_aux[indice_menor_energia]

    objeto_resultado_simulador = resultado_simulador.min_eigen_solver_result

    amostras_otimizacao_simulador = resultado_simulador.samples
    valor_otimo_encontrado_simulador = objeto_resultado_simulador.optimal_value
    tempo_elapsado_otimizador_simulador = objeto_resultado_simulador.optimizer_time
    probabilidade_valor_otimo_simulador = objeto_resultado_simulador.best_measurement['probability']

    resultados_qaoa_4.at[indice, 'parametros_simulador'] = parametros_simulador
    resultados_qaoa_4.at[indice, 'amostras_otimizacao_simulador'] = amostras_otimizacao_simulador
    resultados_qaoa_4.at[indice, 'valor_otimo_encontrado_simulador'] = valor_otimo_encontrado_simulador
    resultados_qaoa_4.at[indice, 'tempo_elapsado_otimizador_simulador'] = tempo_elapsado_otimizador_simulador
    resultados_qaoa_4.at[indice, 'probabilidade_valor_otimo_simulador'] = probabilidade_valor_otimo_simulador

    with open('pickles/resultados_qaoa_4_diferentes_metodos_parcial_s.pickle', 'wb') as file:
            pickle.dump(resultados_qaoa_4, file)

# for indice, row in resultados_qaoa_4.iterrows():
#     try:
#         print(indice)
#         mixer = mixers[indice[1]]
#         otimizador = otimizadores[indice[0]]
#         parametros_iniciais = row['parametros']

#         lista_energia_aux = []
#         lista_parametros_aux = []

#         def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
#             lista_parametros_aux.append(parameters)
#             lista_energia_aux.append(mean)
#             # clear_output(wait=False)

#         qaoa = QAOA(sampler=sampler_computador_quantico, optimizer=otimizador, reps=1, mixer=mixer, initial_point=parametros_iniciais, callback=guardar_parametros_chamada_custo)
#         qaoa_eigen = MinimumEigenOptimizer(qaoa)
#         resultado_computador_quantico = qaoa_eigen.solve(programa_quadratico_4)

#         indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
#         parametros_computador_quantico = lista_parametros_aux[indice_menor_energia]

#         objeto_resultado_computador_quantico = resultado_computador_quantico.min_eigen_solver_result

#         amostras_otimizacao_computador_quantico = resultado_computador_quantico.samples
#         valor_otimo_encontrado_computador_quantico = objeto_resultado_computador_quantico.optimal_value
#         tempo_elapsado_otimizador_computador_quantico = objeto_resultado_computador_quantico.optimizer_time
#         probabilidade_valor_otimo_computador_quantico = objeto_resultado_computador_quantico.best_measurement['probability']

#         resultados_qaoa_4.at[indice, 'parametros_computador_quantico'] = parametros_computador_quantico
#         resultados_qaoa_4.at[indice, 'amostras_otimizacao_computador_quantico'] = amostras_otimizacao_computador_quantico
#         resultados_qaoa_4.at[indice, 'valor_otimo_encontrado_computador_quantico'] = valor_otimo_encontrado_computador_quantico
#         resultados_qaoa_4.at[indice, 'tempo_elapsado_otimizador_computador_quantico'] = tempo_elapsado_otimizador_computador_quantico
#         resultados_qaoa_4.at[indice, 'probabilidade_valor_otimo_computador_quantico'] = probabilidade_valor_otimo_computador_quantico

#         with open('pickles/resultados_qaoa_4_diferentes_metodos_parcial_q.pickle', 'wb') as file:
#             pickle.dump(resultados_qaoa_4, file)
#     except Exception:
#         pass

with open('pickles/resultados_qaoa_4_diferentes_metodos.pickle', 'wb') as file:
    pickle.dump(resultados_qaoa_4, file)

resultados_qaoa_4

,,parametros,valor_otimo_encontrado,tempo_elapsado_otimizador,probabilidade_valor_otimo,amostras_otimizacao,parametros_simulador,amostras_otimizacao_simulador,valor_otimo_encontrado_simulador,tempo_elapsado_otimizador_simulador,probabilidade_valor_otimo_simulador,parametros_computador_quantico,amostras_otimizacao_computador_quantico,valor_otimo_encontrado_computador_quantico,tempo_elapsado_otimizador_computador_quantico,probabilidade_valor_otimo_computador_quantico
otimizador,mixer,,,,,,,,,,,,,,,
COBYLA,full_mixer,"[7.200935379710995, -5.905469512861117]",-0.075302,0.182393,0.108398,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[6.950766609476071, -3.9372672682865937]","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-0.109863,234.178507,0.132,None,None,None,None,None
L_BFGS_B,full_mixer,"[-2.193283203540099, -5.919583952905944]",0.008450,0.318933,0.150391,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[-449674.3328116764, 151869.33555812057]","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-0.008272,1087.595171,0.1345,None,None,None,None,None
NFT,full_mixer,"[220.56057128564484, 80.24507270495481]",0.174098,4.842583,0.141602,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[220.57808042325766, 80.51134141229753]","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-0.11365,1780.707663,0.13425,None,None,None,None,None
SLSQP,full_mixer,"[12881064.979279414, -9012350.491580186]",-0.057253,0.711714,0.126953,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[12882550.673901353, -9007836.8915903]","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",0.043048,1250.294688,0.118,None,None,None,None,None
COBYLA,mixer_default,"[3.3654652812587003, -3.849155850623404]",-0.775968,0.229233,0.121094,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[3.349459644731907, -3.8759558549254542]","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-0.701208,225.77939,0.1155,None,None,None,None,None
L_BFGS_B,mixer_default,"[3.3600878034143027, 3.184892227968962]",-0.787206,0.320539,0.100586,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[3.360087659196336, 3.184892283319375]","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-0.785949,619.055864,0.09725,None,None,None,None,None
NFT,mixer_default,"[82.37547968649578, 40.91973425342345]",0.033338,5.459131,0.900391,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[82.37547968649578, 40.91973425342345]","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-0.896637,1783.69381,0.88675,None,None,None,None,None
SLSQP,mixer_default,"[-1441015.8589012334, -5190740.957774206]",0.406204,0.652428,0.242188,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[-1441015.8589012334, -5190740.957774206]","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",0.054474,1028.785077,0.1845,None,None,None,None,None
COBYLA,parity_ring_mixer,"[5.6291616924156855, 0.23476082961507866]",-0.907475,0.160004,0.083984,"[SolutionSample(x=array([1., 0., 1., 0.]), fva...","[5.601002693536092, 0.21314218148034006]","[SolutionSample(x=array([1., 0., 1., 0.]), fva...",-0.883132,270.623421,0.07025,None,None,None,None,None


In [15]:
copia_resultados_qaoa_4 = resultados_qaoa_4.copy()

combinacoes = [f"{otimizador} e {mixer}" for otimizador, mixer in copia_resultados_qaoa_4.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=False, shared_yaxes=False)

for i, (otimizador, mixer) in enumerate(copia_resultados_qaoa_4.index.unique()):

    df_subset = copia_resultados_qaoa_4.loc[
        (copia_resultados_qaoa_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_qaoa_4.index.get_level_values(1) == mixer)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{mixer}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/qaoa_4_comparacao_metodos.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [16]:

copia_resultados_qaoa_4 = resultados_qaoa_4.copy()

combinacoes = [f"{otimizador} e {mixer}" for otimizador, mixer in copia_resultados_qaoa_4.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=True, shared_yaxes=True,)

for i, (otimizador, mixer) in enumerate(copia_resultados_qaoa_4.index.unique()):

    df_subset = copia_resultados_qaoa_4.loc[
        (copia_resultados_qaoa_4.index.get_level_values(0) == otimizador) &
        (copia_resultados_qaoa_4.index.get_level_values(1) == mixer)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{mixer}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/qaoa_4_comparacao_metodos_mesmo_y.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

## Roda para 10 qubits

In [ ]:
num_qubits = 10

In [ ]:
# Function to create a ring mixer
def create_ring_mixer(num_qubits):
    ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        ring_mixer *= Pauli(''.join(pauli_list))
    return ring_mixer

# Function to create a parity ring mixer
def create_parity_ring_mixer(num_qubits):
    parity_ring_mixer = Pauli('I'*num_qubits)
    for i in range(0, num_qubits, 2):
        pauli_list = list('I'*num_qubits)
        pauli_list[i] = 'X'
        parity_ring_mixer *= Pauli(''.join(pauli_list))
    pauli_list[-1] = 'X'  # Apply X gate on the last qubit
    parity_ring_mixer *= Pauli(''.join(pauli_list))
    return parity_ring_mixer

# Function to create a full mixer
def create_full_mixer(num_qubits):
    return Pauli('X'*num_qubits)

ring_mixer = create_ring_mixer(num_qubits)
full_mixer = create_full_mixer(num_qubits)
parity_ring_mixer = create_parity_ring_mixer(num_qubits)

otimizadores = {'NFT': NFT(maxiter=100),
                'SLSQP': SLSQP(maxiter=100),
                'COBYLA': COBYLA(maxiter=100),
                'L_BFGS_B': L_BFGS_B(maxiter=100)}

mixers = {'mixer_default': None, 
            'ring_mixer': ring_mixer,
            'full_mixer': full_mixer,
            'parity_ring_mixer': parity_ring_mixer}

ansatzes = {'EfficientSU2': EfficientSU2(num_qubits),
            'RealAmplitudes': RealAmplitudes(num_qubits),
            'PauliTwoDesign': PauliTwoDesign(num_qubits, reps=3),
            'TwoLocal': TwoLocal(num_qubits, "ry", "cz", reps=3, entanglement="full")}

## VQE

In [ ]:
resultados_vqe_10['parametros_simulador'] = None
resultados_vqe_10['amostras_otimizacao_simulador'] = None
resultados_vqe_10['valor_otimo_encontrado_simulador'] = None
resultados_vqe_10['tempo_elapsado_otimizador_simulador'] = None
resultados_vqe_10['probabilidade_valor_otimo_simulador'] = None

resultados_vqe_10['parametros_computador_quantico'] = None
resultados_vqe_10['amostras_otimizacao_computador_quantico'] = None
resultados_vqe_10['valor_otimo_encontrado_computador_quantico'] = None
resultados_vqe_10['tempo_elapsado_otimizador_computador_quantico'] = None
resultados_vqe_10['probabilidade_valor_otimo_computador_quantico'] = None

for indice, row in resultados_vqe_10.iterrows():

    ansatz = ansatzes[indice[1]]
    otimizador = otimizadores[indice[0]]
    parametros_iniciais = row['parametros']

    lista_energia_aux = []
    lista_parametros_aux = []

    def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
        lista_parametros_aux.append(parameters)
        lista_energia_aux.append(mean)
        time.sleep(5)
        clear_output(wait=False)

    vqe_simulador = SamplingVQE(sampler=sampler_simulador, ansatz=ansatz, optimizer=otimizador, initial_point=parametros_iniciais, callback=guardar_parametros_chamada_custo)
    vqe_simulador.random_seed = seed
    vqe_eigen_simulador = MinimumEigenOptimizer(vqe_simulador)
    resultado_simulador = vqe_eigen_simulador.solve(programa_quadratico_10)

    indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
    parametros_simulador = lista_parametros_aux[indice_menor_energia]

    objeto_resultado_simulador = resultado_simulador.min_eigen_solver_result

    amostras_otimizacao_simulador = resultado_simulador.samples
    valor_otimo_encontrado_simulador = objeto_resultado_simulador.optimal_value
    tempo_elapsado_otimizador_simulador = objeto_resultado_simulador.optimizer_time
    probabilidade_valor_otimo_simulador = objeto_resultado_simulador.best_measurement['probability']

    resultados_vqe_10.at[indice, 'parametros_simulador'] = parametros_simulador
    resultados_vqe_10.at[indice, 'amostras_otimizacao_simulador'] = amostras_otimizacao_simulador
    resultados_vqe_10.at[indice, 'valor_otimo_encontrado_simulador'] = valor_otimo_encontrado_simulador
    resultados_vqe_10.at[indice, 'tempo_elapsado_otimizador_simulador'] = tempo_elapsado_otimizador_simulador
    resultados_vqe_10.at[indice, 'probabilidade_valor_otimo_simulador'] = probabilidade_valor_otimo_simulador

    with open('pickles/resultados_vqe_10_diferentes_metodos_parcial_s.pickle', 'wb') as file:
        pickle.dump(resultados_vqe_10, file)

# for indice, row in resultados_vqe_10.iterrows():
#     try:
#         ansatz = ansatzes[indice[1]]
#         otimizador = otimizadores[indice[0]]
#         parametros_iniciais = row['parametros']

#         lista_energia_aux = []
#         lista_parametros_aux = []

#         def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
#             lista_parametros_aux.append(parameters)
#             lista_energia_aux.append(mean)
#             clear_output(wait=False)

#         vqe_computador_quantico = SamplingVQE(sampler=sampler_computador_quantico, ansatz=ansatz, optimizer=otimizador, initial_point=parametros_iniciais, callback=guardar_parametros_chamada_custo)
#         vqe_computador_quantico.random_seed = seed
#         vqe_eigen_computador_quantico = MinimumEigenOptimizer(vqe_computador_quantico)
#         resultado_computador_quantico = vqe_eigen_computador_quantico.solve(programa_quadratico_10)

#         indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
#         parametros_computador_quantico = lista_parametros_aux[indice_menor_energia]

#         objeto_resultado_computador_quantico = resultado_computador_quantico.min_eigen_solver_result

#         amostras_otimizacao_computador_quantico = resultado_computador_quantico.samples
#         valor_otimo_encontrado_computador_quantico = objeto_resultado_computador_quantico.optimal_value
#         tempo_elapsado_otimizador_computador_quantico = objeto_resultado_computador_quantico.optimizer_time
#         probabilidade_valor_otimo_computador_quantico = objeto_resultado_computador_quantico.best_measurement['probability']

#         resultados_vqe_10.at[indice, 'parametros_computador_quantico'] = parametros_computador_quantico
#         resultados_vqe_10.at[indice, 'amostras_otimizacao_computador_quantico'] = amostras_otimizacao_computador_quantico
#         resultados_vqe_10.at[indice, 'valor_otimo_encontrado_computador_quantico'] = valor_otimo_encontrado_computador_quantico
#         resultados_vqe_10.at[indice, 'tempo_elapsado_otimizador_computador_quantico'] = tempo_elapsado_otimizador_computador_quantico
#         resultados_vqe_10.at[indice, 'probabilidade_valor_otimo_computador_quantico'] = probabilidade_valor_otimo_computador_quantico

#         with open('pickles/resultados_vqe_10_diferentes_metodos_parcial.pickle_q', 'wb') as file:
#             pickle.dump(resultados_vqe_10, file)

#     except Exception:
#         pass

with open('pickles/resultados_vqe_10_diferentes_metodos.pickle', 'wb') as file:
    pickle.dump(resultados_vqe_10, file)

resultados_vqe_10

In [ ]:
copia_resultados_vqe_10 = resultados_vqe_10.copy()

combinacoes = [f"{otimizador} e {ansatz}" for otimizador, ansatz in copia_resultados_vqe_10.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=False, shared_yaxes=False)

for i, (otimizador, ansatz) in enumerate(copia_resultados_vqe_10.index.unique()):

    df_subset = copia_resultados_vqe_10.loc[
        (copia_resultados_vqe_10.index.get_level_values(0) == otimizador) &
        (copia_resultados_vqe_10.index.get_level_values(1) == ansatz)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador', 'valor_otimo_encontrado_computador_quantico']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{ansatz}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/vqe_10_comparacao_metodos.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [ ]:
copia_resultados_vqe_10 = resultados_vqe_10.copy()

combinacoes = [f"{otimizador} e {ansatz}" for otimizador, ansatz in copia_resultados_vqe_10.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=True, shared_yaxes=True)

for i, (otimizador, ansatz) in enumerate(copia_resultados_vqe_10.index.unique()):

    df_subset = copia_resultados_vqe_10.loc[
        (copia_resultados_vqe_10.index.get_level_values(0) == otimizador) &
        (copia_resultados_vqe_10.index.get_level_values(1) == ansatz)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador', 'valor_otimo_encontrado_computador_quantico']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{ansatz}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/vqe_10_comparacao_metodos_mesmo_y.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

## QAOA

In [ ]:
resultados_qaoa_10['parametros_simulador'] = None
resultados_qaoa_10['amostras_otimizacao_simulador'] = None
resultados_qaoa_10['valor_otimo_encontrado_simulador'] = None
resultados_qaoa_10['tempo_elapsado_otimizador_simulador'] = None
resultados_qaoa_10['probabilidade_valor_otimo_simulador'] = None

resultados_qaoa_10['parametros_computador_quantico'] = None
resultados_qaoa_10['amostras_otimizacao_computador_quantico'] = None
resultados_qaoa_10['valor_otimo_encontrado_computador_quantico'] = None
resultados_qaoa_10['tempo_elapsado_otimizador_computador_quantico'] = None
resultados_qaoa_10['probabilidade_valor_otimo_computador_quantico'] = None

for indice, row in resultados_qaoa_10.iterrows():
    try:
        mixer = mixers[indice[1]]
        otimizador = otimizadores[indice[0]]
        parametros_iniciais = row['parametros']

        lista_energia_aux = []
        lista_parametros_aux = []

        def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
            lista_parametros_aux.append(parameters)
            lista_energia_aux.append(mean)
            time.sleep(5)
            clear_output(wait=False)

        qaoa = QAOA(sampler=sampler_simulador, optimizer=otimizador, reps=1, mixer=mixer, initial_point=parametros_iniciais, callback=guardar_parametros_chamada_custo)
        qaoa_eigen = MinimumEigenOptimizer(qaoa)
        resultado_simulador = qaoa_eigen.solve(programa_quadratico_10)

        indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
        parametros_simulador = lista_parametros_aux[indice_menor_energia]

        objeto_resultado_simulador = resultado_simulador.min_eigen_solver_result

        amostras_otimizacao_simulador = resultado_simulador.samples
        valor_otimo_encontrado_simulador = objeto_resultado_simulador.optimal_value
        tempo_elapsado_otimizador_simulador = objeto_resultado_simulador.optimizer_time
        probabilidade_valor_otimo_simulador = objeto_resultado_simulador.best_measurement['probability']

        resultados_qaoa_10.at[indice, 'parametros_simulador'] = parametros_simulador
        resultados_qaoa_10.at[indice, 'amostras_otimizacao_simulador'] = amostras_otimizacao_simulador
        resultados_qaoa_10.at[indice, 'valor_otimo_encontrado_simulador'] = valor_otimo_encontrado_simulador
        resultados_qaoa_10.at[indice, 'tempo_elapsado_otimizador_simulador'] = tempo_elapsado_otimizador_simulador
        resultados_qaoa_10.at[indice, 'probabilidade_valor_otimo_simulador'] = probabilidade_valor_otimo_simulador

        with open('pickles/resultados_qaoa_10_diferentes_metodos_parcial_s.pickle', 'wb') as file:
            pickle.dump(resultados_qaoa_10, file)

    except Exception:
        pass

# for indice, row in resultados_qaoa_10.iterrows():

#     mixer = mixers[indice[1]]
#     otimizador = otimizadores[indice[0]]
#     parametros_iniciais = row['parametros']

#     lista_energia_aux = []
#     lista_parametros_aux = []

#     def guardar_parametros_chamada_custo(eval_count, parameters, mean, std):
#         lista_parametros_aux.append(parameters)
#         lista_energia_aux.append(mean)
#         clear_output(wait=False)

#     qaoa = QAOA(sampler=sampler_computador_quantico, optimizer=otimizador, reps=1, mixer=mixer, initial_point=parametros_iniciais, callback=guardar_parametros_chamada_custo)
#     qaoa_eigen = MinimumEigenOptimizer(qaoa)
#     resultado_computador_quantico = qaoa_eigen.solve(programa_quadratico_10)

#     indice_menor_energia = lista_energia_aux.index(min(lista_energia_aux))
#     parametros_computador_quantico = lista_parametros_aux[indice_menor_energia]

#     objeto_resultado_computador_quantico = resultado_computador_quantico.min_eigen_solver_result

#     amostras_otimizacao_computador_quantico = resultado_computador_quantico.samples
#     valor_otimo_encontrado_computador_quantico = objeto_resultado_computador_quantico.optimal_value
#     tempo_elapsado_otimizador_computador_quantico = objeto_resultado_computador_quantico.optimizer_time
#     probabilidade_valor_otimo_computador_quantico = objeto_resultado_computador_quantico.best_measurement['probability']

#     resultados_qaoa_10.at[indice, 'parametros_computador_quantico'] = parametros_computador_quantico
#     resultados_qaoa_10.at[indice, 'amostras_otimizacao_computador_quantico'] = amostras_otimizacao_computador_quantico
#     resultados_qaoa_10.at[indice, 'valor_otimo_encontrado_computador_quantico'] = valor_otimo_encontrado_computador_quantico
#     resultados_qaoa_10.at[indice, 'tempo_elapsado_otimizador_computador_quantico'] = tempo_elapsado_otimizador_computador_quantico
#     resultados_qaoa_10.at[indice, 'probabilidade_valor_otimo_computador_quantico'] = probabilidade_valor_otimo_computador_quantico

#     with open('pickles/resultados_qaoa_10_diferentes_metodos_parcial_q.pickle', 'wb') as file:
#         pickle.dump(resultados_qaoa_10, file)

with open('pickles/resultados_qaoa_10_diferentes_metodos.pickle', 'wb') as file:
    pickle.dump(resultados_qaoa_10, file)

resultados_qaoa_10

In [ ]:
copia_resultados_qaoa_10 = resultados_qaoa_10.copy()

combinacoes = [f"{otimizador} e {mixer}" for otimizador, mixer in copia_resultados_qaoa_10.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=False, shared_yaxes=False)

for i, (otimizador, mixer) in enumerate(copia_resultados_qaoa_10.index.unique()):

    df_subset = copia_resultados_qaoa_10.loc[
        (copia_resultados_qaoa_10.index.get_level_values(0) == otimizador) &
        (copia_resultados_qaoa_10.index.get_level_values(1) == mixer)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador', 'valor_otimo_encontrado_computador_quantico']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{mixer}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/qaoa_10_comparacao_metodos.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

In [ ]:

copia_resultados_qaoa_10 = resultados_qaoa_10.copy()

combinacoes = [f"{otimizador} e {mixer}" for otimizador, mixer in copia_resultados_qaoa_10.index.unique()]
cores = {'valor_otimo_encontrado': 'blue', 'valor_otimo_encontrado_simulador': 'orange', 'valor_otimo_encontrado_computador_quantico':'cyan'}
nomes = {'valor_otimo_encontrado': 'simulador local', 'valor_otimo_encontrado_simulador': 'simulador QASM', 'valor_otimo_encontrado_computador_quantico':'IBM Brisbane'}
cols_appeared = []

fig = make_subplots(rows=4, cols=4, subplot_titles=combinacoes, 
                    x_title='Métodos de execução', y_title='Energia', 
                    shared_xaxes=True, shared_yaxes=True,)

for i, (otimizador, mixer) in enumerate(copia_resultados_qaoa_10.index.unique()):

    df_subset = copia_resultados_qaoa_10.loc[
        (copia_resultados_qaoa_10.index.get_level_values(0) == otimizador) &
        (copia_resultados_qaoa_10.index.get_level_values(1) == mixer)
    ]

    valores = df_subset[['valor_otimo_encontrado', 'valor_otimo_encontrado_simulador', 'valor_otimo_encontrado_computador_quantico']]

    for idx, col in enumerate(valores.columns):
        nome_fig = f'{otimizador}-{mixer}-{col}'
        legenda_bool = False

        if not col in cols_appeared:
            nome_fig = f'{nomes[col]}'
            legenda_bool = True
        
        fig.add_trace(
            go.Bar(x=[col], 
                    y=[valores[col].values[0]],
                    name=nome_fig,
                    marker_color=cores[col],
                    showlegend=legenda_bool),
            row=(i // 4) + 1,
            col=(i % 4) + 1
        )
        cols_appeared.append(col)

        fig.update_xaxes(showticklabels=False, title_text='', row=(i // 4) + 1, col=(i % 4) + 1)

fig.update_layout(
                  showlegend=True,
                  xaxis_title='',
                  yaxis_title='',
                  width = 2200,
                  height = 700,
                  font = dict(size=20),
                  legend_itemsizing="constant",
                  margin = dict(l=35, r=35, t=35, b=75),
                  )

fig.update_annotations(font_size=20)

nome_figura = f"figuras_dados/qaoa_10_comparacao_metodos_mesmo_y.png"
fig.write_image(nome_figura, width=2200, height=700)

fig.show()

# Plots circuitos

Circuitos mixers

In [ ]:
from qiskit import QuantumCircuit

def create_ring_mixer_display():
    ring_mixer = QuantumCircuit(4)
    for i in range(0, 4, 2):
        ring_mixer.x(i)
    return ring_mixer

def create_parity_ring_mixer_display():
    parity_ring_mixer = QuantumCircuit(4)
    for i in range(0, 4, 2):
        parity_ring_mixer.x(i)
    parity_ring_mixer.x(4 - 1)
    return parity_ring_mixer

def create_full_mixer_display():
    full_mixer = QuantumCircuit(4)
    for qubit in range(4):
        full_mixer.x(qubit)
    return full_mixer

In [ ]:
create_ring_mixer_display().draw('mpl')

In [ ]:
create_parity_ring_mixer_display().draw('mpl')

In [ ]:
create_full_mixer_display().draw('mpl')

Circuitos ansatzes

In [ ]:
EfficientSU2(4).decompose().draw('mpl')

In [ ]:
RealAmplitudes(4).decompose().draw('mpl')

In [ ]:
PauliTwoDesign(4, reps=3).decompose().draw('mpl')

In [ ]:
TwoLocal(4, "ry", "cz", reps=3, entanglement="full").decompose().draw('mpl')